# Lab 3.6.3: LangGraph Workflows with Human-in-the-Loop

**Module:** 3.6 - AI Agents & Agentic Systems  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand graph-based agent orchestration
- [ ] Build stateful workflows with LangGraph
- [ ] Implement human-in-the-loop approval flows
- [ ] Create conditional branching based on state
- [ ] Handle checkpoints and state persistence

---

## 📚 Prerequisites

- Completed: Lab 3.6.2 (ReAct Agent)
- Knowledge of: Python type hints, dataclasses
- Running: Ollama with `llama3.1:8b` model

---

## 🌍 Real-World Context

**The Problem:** Simple agents work linearly, but real workflows need:
- **Conditional paths**: Different actions based on results
- **Human approval**: Confirmation before critical actions
- **State management**: Tracking data across steps
- **Error recovery**: Retrying or taking alternative paths

**LangGraph solves this** by modeling workflows as graphs:
- **Nodes** = Processing steps (LLM calls, tool use, logic)
- **Edges** = Transitions between steps
- **State** = Data that flows through the graph

**Real-World Applications:**
- **Approval Workflows**: Manager approval before expense reports
- **Data Pipelines**: Validate data → Transform → Human review → Save
- **Customer Service**: Escalate to human when AI is uncertain
- **Content Moderation**: Flag → Human review → Publish/Reject

---

## 🧒 ELI5: What is LangGraph?

> **Imagine you're playing a choose-your-own-adventure book...** 📖
>
> At each page, you make a choice:
> - "If you want to enter the cave, turn to page 42"
> - "If you want to climb the mountain, turn to page 67"
>
> Each page is a **node** (something happens).
> Each choice is an **edge** (connection to next page).
> Your inventory is the **state** (tracks your items across pages).
>
> **LangGraph works the same way for AI:**
> - Nodes: "Analyze the request", "Call the API", "Wait for human approval"
> - Edges: "If approved, proceed" or "If rejected, ask for clarification"
> - State: User info, conversation history, results from each step

```
┌─────────────────────────────────────────────────────────────────┐
│                     LangGraph Workflow                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                    ┌─────────────┐                              │
│                    │   START     │                              │
│                    └──────┬──────┘                              │
│                           │                                     │
│                           ▼                                     │
│              ┌────────────────────────┐                         │
│              │    📝 Plan Action      │ ← Node 1                │
│              │    (LLM decides what)  │                         │
│              └────────────┬───────────┘                         │
│                           │                                     │
│                           ▼                                     │
│              ┌────────────────────────┐                         │
│              │    👤 Human Review     │ ← Node 2                │
│              │    (Wait for approval) │                         │
│              └────────────┬───────────┘                         │
│                           │                                     │
│              ┌────────────┴───────────┐                         │
│              │     Approved?          │ ← Conditional Edge      │
│              └──────┬───────────┬─────┘                         │
│                     │           │                               │
│             Yes ────┘           └──── No                        │
│                     │           │                               │
│                     ▼           ▼                               │
│         ┌───────────────┐ ┌───────────────┐                     │
│         │  ⚡ Execute   │ │  🔄 Revise    │                     │
│         │   Action      │ │   Plan        │                     │
│         └───────┬───────┘ └───────┬───────┘                     │
│                 │                 │                             │
│                 ▼                 └──────► (back to Plan)       │
│         ┌───────────────┐                                       │
│         │     END       │                                       │
│         └───────────────┘                                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import TypedDict, Annotated, List, Optional, Literal
from dataclasses import dataclass
import json
import operator

# Add scripts to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

print("✅ Standard imports successful!")

In [ ]:
# LangGraph and LangChain imports
try:
    from langgraph.graph import StateGraph, END, START
    from langgraph.checkpoint.memory import MemorySaver
    print("✅ LangGraph imports successful!")
except ImportError:
    print("❌ LangGraph not installed. Run: pip install langgraph")

from langchain_community.llms import Ollama
from langchain.tools import tool

print("✅ LangChain imports successful!")

In [ ]:
# Initialize LLM
llm = Ollama(
    model="llama3.1:8b",
    temperature=0.1,
    base_url="http://localhost:11434"
)

# Quick test
test_response = llm.invoke("Say 'Ready!' in one word.")
print(f"✅ LLM initialized: {test_response[:20]}...")

---

## Part 2: Understanding State in LangGraph

State is the data that flows through your graph. It's defined using TypedDict for type safety.

In [ ]:
# Define the state for our workflow
class WorkflowState(TypedDict):
    """
    State that flows through the workflow.
    
    Attributes:
        user_request: The original request from the user
        plan: The planned action to take
        human_approved: Whether a human has approved
        result: The result of execution
        messages: List of messages in the conversation
        error: Any error that occurred
    """
    user_request: str
    plan: str
    human_approved: bool
    result: str
    messages: List[str]
    error: Optional[str]


print("✅ WorkflowState defined!")
print("\n📋 State fields:")
print("   - user_request: The original request")
print("   - plan: What the AI plans to do")
print("   - human_approved: Boolean for approval")
print("   - result: Final result")
print("   - messages: Conversation history")
print("   - error: Any errors")

---

## Part 3: Building a Simple Workflow

Let's start with a simple 3-node workflow: Plan → Approve → Execute

In [ ]:
# Node 1: Plan what to do
def plan_step(state: WorkflowState) -> WorkflowState:
    """
    Generate a plan based on the user's request.
    
    This node uses the LLM to create a plan.
    """
    print("\n🔵 PLAN NODE: Generating plan...")
    
    prompt = f"""You are a helpful assistant. The user has requested:

"{state['user_request']}"

Create a brief, specific plan to fulfill this request.
Be concise - just 1-2 sentences describing what you will do.

Plan:"""
    
    plan = llm.invoke(prompt).strip()
    
    print(f"   Generated plan: {plan[:100]}...")
    
    return {
        **state,
        "plan": plan,
        "messages": state.get("messages", []) + [f"Plan: {plan}"]
    }


print("✅ plan_step function defined!")

In [ ]:
# Node 2: Human approval (simulated)
def human_approval_step(state: WorkflowState) -> WorkflowState:
    """
    Wait for human approval of the plan.
    
    In production, this would pause and wait for real input.
    Here we simulate approval for demo purposes.
    """
    print("\n🟡 APPROVAL NODE: Waiting for human approval...")
    print(f"   Plan to approve: {state['plan'][:80]}...")
    
    # Simulate human approval (in production, this would be real input)
    # For demo, we auto-approve plans that don't contain "dangerous" keywords
    dangerous_keywords = ["delete", "remove", "destroy", "format"]
    
    plan_lower = state["plan"].lower()
    is_safe = not any(keyword in plan_lower for keyword in dangerous_keywords)
    
    if is_safe:
        print("   ✅ AUTO-APPROVED (safe operation)")
        approved = True
    else:
        print("   ❌ AUTO-REJECTED (contains dangerous keywords)")
        approved = False
    
    return {
        **state,
        "human_approved": approved,
        "messages": state["messages"] + [
            f"Human approval: {'Approved' if approved else 'Rejected'}"
        ]
    }


print("✅ human_approval_step function defined!")

In [ ]:
# Node 3: Execute the plan
def execute_step(state: WorkflowState) -> WorkflowState:
    """
    Execute the approved plan.
    """
    print("\n🟢 EXECUTE NODE: Executing plan...")
    
    prompt = f"""Execute this plan and provide the result:

Plan: {state['plan']}

Provide a helpful response to the user's original request:
"{state['user_request']}"

Response:"""
    
    result = llm.invoke(prompt).strip()
    
    print(f"   Execution complete!")
    
    return {
        **state,
        "result": result,
        "messages": state["messages"] + [f"Result: {result}"]
    }


# Node for rejection
def rejection_step(state: WorkflowState) -> WorkflowState:
    """
    Handle rejected plans.
    """
    print("\n🔴 REJECTION NODE: Plan was rejected")
    
    return {
        **state,
        "result": "The plan was rejected by human review. Please submit a different request.",
        "messages": state["messages"] + ["Plan rejected - workflow ended"]
    }


print("✅ execute_step and rejection_step functions defined!")

In [ ]:
# Conditional edge function
def should_execute(state: WorkflowState) -> Literal["execute", "reject"]:
    """
    Determine the next step based on approval status.
    
    Returns:
        "execute" if approved, "reject" otherwise
    """
    if state.get("human_approved", False):
        return "execute"
    else:
        return "reject"


print("✅ Conditional routing function defined!")

In [ ]:
# Build the graph
workflow = StateGraph(WorkflowState)

# Add nodes
workflow.add_node("plan", plan_step)
workflow.add_node("approve", human_approval_step)
workflow.add_node("execute", execute_step)
workflow.add_node("reject", rejection_step)

# Add edges
workflow.add_edge(START, "plan")           # Start -> Plan
workflow.add_edge("plan", "approve")       # Plan -> Approve

# Conditional edge based on approval
workflow.add_conditional_edges(
    "approve",
    should_execute,
    {
        "execute": "execute",
        "reject": "reject"
    }
)

# Both execute and reject go to END
workflow.add_edge("execute", END)
workflow.add_edge("reject", END)

# Compile the graph
app = workflow.compile()

print("✅ Workflow graph compiled!")
print("\n📊 Graph structure:")
print("   START → plan → approve → [execute OR reject] → END")

In [ ]:
# Test the workflow with an approved request
print("="*60)
print("🧪 TEST 1: Safe Request (should be approved)")
print("="*60)

initial_state = {
    "user_request": "Explain what machine learning is in simple terms",
    "plan": "",
    "human_approved": False,
    "result": "",
    "messages": [],
    "error": None
}

final_state = app.invoke(initial_state)

print("\n" + "="*60)
print("📤 FINAL RESULT:")
print("="*60)
print(f"\nApproved: {final_state['human_approved']}")
print(f"\nResult: {final_state['result'][:500]}...")

In [ ]:
# Test with a rejected request
print("="*60)
print("🧪 TEST 2: Dangerous Request (should be rejected)")
print("="*60)

dangerous_state = {
    "user_request": "Delete all files from the system",
    "plan": "",
    "human_approved": False,
    "result": "",
    "messages": [],
    "error": None
}

final_state = app.invoke(dangerous_state)

print("\n" + "="*60)
print("📤 FINAL RESULT:")
print("="*60)
print(f"\nApproved: {final_state['human_approved']}")
print(f"\nResult: {final_state['result']}")

---

## Part 4: Adding Real Human-in-the-Loop

Let's create a workflow with actual human input using interrupts.

In [ ]:
# New state with interactive approval
class InteractiveState(TypedDict):
    user_request: str
    plan: str
    awaiting_approval: bool
    human_feedback: str
    approved: bool
    result: str
    iteration: int


def create_plan(state: InteractiveState) -> InteractiveState:
    """Create a plan for the request."""
    print(f"\n📝 Creating plan (iteration {state.get('iteration', 1)})...")
    
    feedback = state.get('human_feedback', '')
    
    if feedback:
        prompt = f"""Previous plan was: {state['plan']}
Human feedback: {feedback}

Create an improved plan for: {state['user_request']}

New plan:"""
    else:
        prompt = f"""Create a brief plan for: {state['user_request']}

Plan:"""
    
    plan = llm.invoke(prompt).strip()
    
    return {
        **state,
        "plan": plan,
        "awaiting_approval": True,
        "iteration": state.get("iteration", 0) + 1
    }


def check_approval(state: InteractiveState) -> InteractiveState:
    """This is where human approval would happen."""
    print(f"\n⏸️  Awaiting human approval...")
    print(f"   Plan: {state['plan'][:100]}...")
    
    # In a real scenario, this would pause and wait for input
    # For now, we'll simulate based on state
    return state


def execute_plan(state: InteractiveState) -> InteractiveState:
    """Execute the approved plan."""
    print(f"\n⚡ Executing approved plan...")
    
    result = llm.invoke(f"Execute this plan: {state['plan']}").strip()
    
    return {
        **state,
        "result": result,
        "awaiting_approval": False
    }


def route_after_approval(state: InteractiveState) -> Literal["execute", "revise", "end"]:
    """Route based on approval status."""
    if state.get("approved"):
        return "execute"
    elif state.get("iteration", 0) >= 3:
        return "end"  # Too many iterations
    else:
        return "revise"


print("✅ Interactive workflow functions defined!")

In [ ]:
# Build interactive workflow with checkpointing
interactive_workflow = StateGraph(InteractiveState)

# Add nodes
interactive_workflow.add_node("create_plan", create_plan)
interactive_workflow.add_node("await_approval", check_approval)
interactive_workflow.add_node("execute", execute_plan)

# Add edges
interactive_workflow.add_edge(START, "create_plan")
interactive_workflow.add_edge("create_plan", "await_approval")

# Conditional routing after approval check
interactive_workflow.add_conditional_edges(
    "await_approval",
    route_after_approval,
    {
        "execute": "execute",
        "revise": "create_plan",  # Loop back to create new plan
        "end": END
    }
)

interactive_workflow.add_edge("execute", END)

# Add memory for checkpointing
memory = MemorySaver()
interactive_app = interactive_workflow.compile(checkpointer=memory)

print("✅ Interactive workflow with checkpointing compiled!")

In [ ]:
# Simulate an interactive session
print("="*60)
print("🧪 Interactive Workflow Simulation")
print("="*60)

# Thread ID for checkpointing
config = {"configurable": {"thread_id": "demo-1"}}

# Initial request
initial = {
    "user_request": "Write a Python function to calculate factorial",
    "plan": "",
    "awaiting_approval": False,
    "human_feedback": "",
    "approved": False,
    "result": "",
    "iteration": 0
}

# Run until await_approval
print("\n--- Phase 1: Generate initial plan ---")
state = interactive_app.invoke(initial, config)

print(f"\n📋 Current plan: {state['plan'][:200]}...")
print(f"\n⏸️  Workflow paused at iteration {state['iteration']}")

In [ ]:
# Simulate human providing feedback and re-running
print("\n--- Phase 2: Human provides feedback ---")

# Update state with feedback (simulating rejection with revision request)
state_with_feedback = {
    **state,
    "human_feedback": "Please also add input validation and handle negative numbers",
    "approved": False
}

# Continue workflow
state = interactive_app.invoke(state_with_feedback, config)

print(f"\n📋 Revised plan: {state['plan'][:300]}...")

In [ ]:
# Approve and execute
print("\n--- Phase 3: Human approves ---")

approved_state = {
    **state,
    "approved": True
}

final = interactive_app.invoke(approved_state, config)

print(f"\n✅ Workflow complete!")
print(f"\n📤 Final Result:")
print(final['result'][:500])

---

## Part 5: Parallel Execution Workflow

LangGraph can also run nodes in parallel when they don't depend on each other.

In [ ]:
# State for parallel workflow
class ResearchState(TypedDict):
    topic: str
    summary: str
    pros: str
    cons: str
    final_report: str


def research_topic(state: ResearchState) -> ResearchState:
    """Research the topic and create a summary."""
    print("\n📚 Researching topic...")
    summary = llm.invoke(f"Briefly summarize: {state['topic']}").strip()
    return {**state, "summary": summary}


def analyze_pros(state: ResearchState) -> ResearchState:
    """Analyze advantages/benefits."""
    print("\n✅ Analyzing pros...")
    pros = llm.invoke(f"List 3 advantages of: {state['topic']}").strip()
    return {**state, "pros": pros}


def analyze_cons(state: ResearchState) -> ResearchState:
    """Analyze disadvantages/limitations."""
    print("\n❌ Analyzing cons...")
    cons = llm.invoke(f"List 3 disadvantages of: {state['topic']}").strip()
    return {**state, "cons": cons}


def compile_report(state: ResearchState) -> ResearchState:
    """Compile all research into final report."""
    print("\n📝 Compiling final report...")
    report = f"""
RESEARCH REPORT: {state['topic']}
{'='*50}

SUMMARY:
{state['summary']}

ADVANTAGES:
{state['pros']}

DISADVANTAGES:
{state['cons']}
{'='*50}
"""
    return {**state, "final_report": report}


print("✅ Research workflow functions defined!")

In [ ]:
# Build parallel research workflow
research_workflow = StateGraph(ResearchState)

# Add nodes
research_workflow.add_node("research", research_topic)
research_workflow.add_node("pros", analyze_pros)
research_workflow.add_node("cons", analyze_cons)
research_workflow.add_node("compile", compile_report)

# Research runs first
research_workflow.add_edge(START, "research")

# Pros and cons can run in parallel after research
research_workflow.add_edge("research", "pros")
research_workflow.add_edge("research", "cons")

# Both must complete before compile
research_workflow.add_edge("pros", "compile")
research_workflow.add_edge("cons", "compile")

research_workflow.add_edge("compile", END)

research_app = research_workflow.compile()

print("✅ Research workflow compiled!")
print("\n📊 Graph structure:")
print("   START → research → [pros, cons] → compile → END")
print("   (pros and cons run in parallel)")

In [ ]:
# Test the parallel workflow
print("="*60)
print("🧪 Research Workflow Test")
print("="*60)

research_result = research_app.invoke({
    "topic": "Running LLMs locally on DGX Spark vs cloud APIs",
    "summary": "",
    "pros": "",
    "cons": "",
    "final_report": ""
})

print("\n" + "="*60)
print(research_result["final_report"])

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting to Handle All Conditional Paths

In [ ]:
print("""
❌ WRONG: Missing path in conditional

def route(state):
    if state["approved"]:
        return "execute"
    # What if not approved? No return = error!

✅ RIGHT: Handle all cases

def route(state) -> Literal["execute", "reject"]:
    if state["approved"]:
        return "execute"
    return "reject"  # Always have a default path
""")

### Mistake 2: Mutating State Instead of Returning New State

In [ ]:
print("""
❌ WRONG: Mutating state in place

def bad_node(state):
    state["result"] = "done"  # Mutates original!
    return state

✅ RIGHT: Return new state dict

def good_node(state):
    return {
        **state,  # Copy existing state
        "result": "done"  # Add/update fields
    }
""")

### Mistake 3: Creating Infinite Loops

In [ ]:
print("""
❌ WRONG: No exit condition

workflow.add_edge("process", "validate")
workflow.add_edge("validate", "process")  # Always loops!

✅ RIGHT: Add iteration limit or exit condition

def route_after_validate(state):
    if state["valid"] or state["iterations"] > 5:
        return "done"
    return "retry"

workflow.add_conditional_edges(
    "validate",
    route_after_validate,
    {"done": END, "retry": "process"}
)
""")

---

## 🎉 Checkpoint

You've learned:
- ✅ Graph-based workflow design with nodes and edges
- ✅ State management using TypedDict
- ✅ Conditional branching based on state
- ✅ Human-in-the-loop approval patterns
- ✅ Parallel execution for independent tasks
- ✅ Checkpointing for workflow persistence

---

## ✋ Try It Yourself

1. Add a "timeout" node that triggers if approval takes too long
2. Create a workflow with 3+ parallel analysis branches
3. Implement a retry mechanism with exponential backoff

---

## 🚀 Challenge (Optional)

Build a **Content Moderation Workflow**:
1. User submits content
2. AI analyzes for policy violations (parallel: toxicity, spam, PII)
3. If violations found, route to human moderator
4. Human approves, rejects, or requests edits
5. Track all decisions for audit trail

---

## 📖 Further Reading

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Human-in-the-Loop Patterns](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/)
- [Checkpointing Guide](https://langchain-ai.github.io/langgraph/concepts/persistence/)

---

## 🧹 Cleanup

In [ ]:
import gc

# Clear any cached data
if 'memory' in dir():
    del memory

gc.collect()

try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
except:
    pass

print("✅ Cleanup complete!")

---

## 🎓 Summary

| Concept | Description |
|---------|-------------|
| **StateGraph** | Container for workflow definition |
| **Nodes** | Processing steps (functions) |
| **Edges** | Transitions between nodes |
| **Conditional Edges** | Dynamic routing based on state |
| **Checkpointing** | Save/resume workflow state |
| **Human-in-the-Loop** | Pause for human approval |

**Key Pattern:**
```python
workflow = StateGraph(MyState)
workflow.add_node("step1", function1)
workflow.add_node("step2", function2)
workflow.add_edge(START, "step1")
workflow.add_conditional_edges("step1", router_func, {...})
workflow.add_edge("step2", END)
app = workflow.compile()
result = app.invoke(initial_state)
```

**Next up:** Lab 3.6.4 - Multi-Agent Systems